In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Normalize, Compose


In [2]:
# Define the transformation
transform = Compose([ToTensor(), Normalize((0.5,), (0.5,))])
# Download the MNIST data
train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9912422/9912422 [00:11<00:00, 845463.32it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28881/28881 [00:00<00:00, 114776.28it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1648877/1648877 [00:07<00:00, 224234.19it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4542/4542 [00:00<00:00, 2911144.37it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [3]:
class MNISTDataset(Dataset):
    def __init__(self, mnist_data):
        self.mnist_data = mnist_data

    def __len__(self):
        return len(self.mnist_data)

    def __getitem__(self, idx):
        image, label = self.mnist_data[idx]
        image = image.view(-1)  # Flatten the image
        return image, label

# Create instances of the custom dataset
train_dataset = MNISTDataset(train_data)
test_dataset = MNISTDataset(test_data)


In [4]:
trainloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [13]:
class DeepNet(nn.Module):
    def __init__(self):
        super(DeepNet, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 10)
        self.leaky_relu = nn.LeakyReLU(negative_slope=0.01)

    def forward(self, x):
        x = self.leaky_relu(self.fc1(x))
        x = self.leaky_relu(self.fc2(x))
        x = self.leaky_relu(self.fc3(x))
        x = self.leaky_relu(self.fc4(x))
        x = self.fc5(x)
        return x

net = DeepNet()
print(net)


DeepNet(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=10, bias=True)
  (leaky_relu): LeakyReLU(negative_slope=0.01)
)


In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters(), lr=0.001, weight_decay=0.01)

In [15]:
for epoch in range(5):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:  # print every 200 mini-batches
            print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 200:.3f}')
            running_loss = 0.0

print('Finished Training')


Epoch 1, Batch 200, Loss: 0.887
Epoch 1, Batch 400, Loss: 0.392
Epoch 1, Batch 600, Loss: 0.315
Epoch 1, Batch 800, Loss: 0.303
Epoch 1, Batch 1000, Loss: 0.240
Epoch 1, Batch 1200, Loss: 0.241
Epoch 1, Batch 1400, Loss: 0.217
Epoch 1, Batch 1600, Loss: 0.218
Epoch 1, Batch 1800, Loss: 0.210
Epoch 2, Batch 200, Loss: 0.169
Epoch 2, Batch 400, Loss: 0.171
Epoch 2, Batch 600, Loss: 0.165
Epoch 2, Batch 800, Loss: 0.148
Epoch 2, Batch 1000, Loss: 0.154
Epoch 2, Batch 1200, Loss: 0.144
Epoch 2, Batch 1400, Loss: 0.156
Epoch 2, Batch 1600, Loss: 0.153
Epoch 2, Batch 1800, Loss: 0.159
Epoch 3, Batch 200, Loss: 0.114
Epoch 3, Batch 400, Loss: 0.115
Epoch 3, Batch 600, Loss: 0.120
Epoch 3, Batch 800, Loss: 0.128
Epoch 3, Batch 1000, Loss: 0.127
Epoch 3, Batch 1200, Loss: 0.145
Epoch 3, Batch 1400, Loss: 0.115
Epoch 3, Batch 1600, Loss: 0.111
Epoch 3, Batch 1800, Loss: 0.106
Epoch 4, Batch 200, Loss: 0.094
Epoch 4, Batch 400, Loss: 0.094
Epoch 4, Batch 600, Loss: 0.107
Epoch 4, Batch 800, Loss:

In [16]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test set: {100 * correct / total:.2f}%')


Accuracy of the network on the test set: 97.74%
